#**IMPORTING LIBRARIES AND DEFINING VARIABLES**

In [1]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense
import keras
import numpy as np

batch_size = 64
epochs = 100
latent_dim = 256
num_samples = 10000

data_path = "fra.txt"

#**Processing the i/p & o/p (please see the print statements to understand)**

In [2]:
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()

with open(data_path, "r", encoding="utf-8") as f:
    lines = f.read().split("\n")
print("Input lines ==>", *[lines[i] for i in range(5)], "....", sep='\n')
print()

# To iterate through first 10K elements in the list
for line in lines[: min(num_samples, len(lines) - 1)]:
  input_text, target_text, source_info = line.split("\t")

  # We use "[START]" as the "start sequence" character
  # for the targets, and "[END]" as "end sequence" character.
  target_text = "[START] " + target_text + " [END]"

  for word in input_text.split():
    input_characters.add(word)
  for word in target_text.split():
    target_characters.add(word)

  input_texts.append(input_text)
  target_texts.append(target_text)

input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
input_characters.append(" ")
target_characters.append(" ")

print("English sent. array ==>", *[input_texts[i] for i in range(5)], "....", sep='\n')
print()

print("French sent. array ==>", *[target_texts[i] for i in range(5)], "....", sep='\n')
print()

print("Unique English characters ==>", input_characters, sep='\n')
print()

print("Unique French characters ==>", target_characters, sep='\n')
print()

Input lines ==>
Go.	Va !	CC-BY 2.0 (France) Attribution: tatoeba.org #2877272 (CM) & #1158250 (Wittydev)
Go.	Marche.	CC-BY 2.0 (France) Attribution: tatoeba.org #2877272 (CM) & #8090732 (Micsmithel)
Go.	En route !	CC-BY 2.0 (France) Attribution: tatoeba.org #2877272 (CM) & #8267435 (felix63)
Go.	Bouge !	CC-BY 2.0 (France) Attribution: tatoeba.org #2877272 (CM) & #9022935 (Micsmithel)
Hi.	Salut !	CC-BY 2.0 (France) Attribution: tatoeba.org #538123 (CM) & #509819 (Aiji)
....

English sent. array ==>
Go.
Go.
Go.
Go.
Hi.
....

French sent. array ==>
[START] Va ! [END]
[START] Marche. [END]
[START] En route ! [END]
[START] Bouge ! [END]
[START] Salut ! [END]
....

Unique English characters ==>
['"aah."', '$300.', '$5.', '17,', '19.', '2:30', '2:30.', '3:30.', '50', '7:30.', '8:30.', '99%', 'A', 'A.', 'Abandon', 'Act', 'After', 'Aim', 'Aim.', "Ain't", 'Air', 'All', 'Am', 'American.', 'Answer', 'Any', 'Anybody', 'Anyone', 'Anything', 'Arabs.', 'Are', 'Arm', 'Asian.', 'Ask', 'Attack!', 'Avoid'

#**Computing important parameters required for generating i/p & o/p in Model**

In [3]:
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt.split()) for txt in input_texts])
max_decoder_seq_length = max([len(txt.split()) for txt in target_texts])

print("Number of samples:", len(input_texts))

print("Number of unique input tokens (encoder tokens):", num_encoder_tokens)
print("Number of unique output tokens (decoder tokens):", num_decoder_tokens)
print()

print("Max sequence length for inputs:", max_encoder_seq_length)
print("Max sequence string for inputs:", input_texts[np.argmax([len(txt) for txt in input_texts])])
print()

print("Max sequence length for outputs:", max_decoder_seq_length)
print("Max sequence string for outputs:", target_texts[np.argmax([len(txt) for txt in target_texts])])

Number of samples: 10000
Number of unique input tokens (encoder tokens): 2862
Number of unique output tokens (decoder tokens): 5596

Max sequence length for inputs: 4
Max sequence string for inputs: A car hit Tom.

Max sequence length for outputs: 12
Max sequence string for outputs: [START] On m'a demandé ma carte d'identité pour vérifier mon âge. [END]


#**Tokenizing the data**

In [4]:
input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])

print("Tokenized dictionary (input):", input_token_index)
print("Tokenized dictionary (output):", target_token_index)
print()

print("Max sized sequence string for inputs:", input_texts[np.argmax([len(txt) for txt in input_texts])])
print([input_token_index[i] for i in input_texts[np.argmax([len(txt) for txt in input_texts])].split()])
print()

print("Max sized sequence string for outputs:", target_texts[np.argmax([len(txt) for txt in target_texts])])
print([target_token_index[i] for i in target_texts[np.argmax([len(txt) for txt in target_texts])].split()])

Tokenized dictionary (input): {'"aah."': 0, '$300.': 1, '$5.': 2, '17,': 3, '19.': 4, '2:30': 5, '2:30.': 6, '3:30.': 7, '50': 8, '7:30.': 9, '8:30.': 10, '99%': 11, 'A': 12, 'A.': 13, 'Abandon': 14, 'Act': 15, 'After': 16, 'Aim': 17, 'Aim.': 18, "Ain't": 19, 'Air': 20, 'All': 21, 'Am': 22, 'American.': 23, 'Answer': 24, 'Any': 25, 'Anybody': 26, 'Anyone': 27, 'Anything': 28, 'Arabs.': 29, 'Are': 30, 'Arm': 31, 'Asian.': 32, 'Ask': 33, 'Attack!': 34, 'Avoid': 35, 'Awesome!': 36, 'Awesome.': 37, 'Back': 38, 'Be': 39, 'Beat': 40, 'Beef,': 41, 'Beer': 42, "Beer's": 43, 'Begin.': 44, 'Beware': 45, 'Birds': 46, 'Bless': 47, 'Blindfold': 48, 'Boston.': 49, 'Bottoms': 50, 'Boys': 51, 'Break': 52, 'Breathe': 53, 'Bring': 54, 'British.': 55, 'Burn': 56, 'Bury': 57, 'Buy': 58, 'Buzz': 59, 'CD.': 60, 'CDs.': 61, 'CPR.': 62, 'Call': 63, 'Calm': 64, 'Can': 65, 'Canadian.': 66, 'Cancel': 67, 'Carry': 68, 'Catch': 69, 'Cats': 70, 'Check': 71, 'Check,': 72, 'Cheer': 73, 'Cheers!': 74, 'Chill': 75, 'Ch

#**Initializing input and output for encoder and decoder**

In [5]:
# The snippets below will create sparse numpy arrays of 3D shape using the parameters given as dimensions of the array.
# np.zeros((2, 3, 4)) generates something like:
# [
#    [
#        [
#            0, 0, 0, 0
#        ] X 4,
#        [
#            0, 0, 0, 0
#        ]... upto 3
#    ] X 3
#] X 2

encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype="float32",
)

decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype="float32",
)

decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype="float32",
)

print("Encoder input:", encoder_input_data[0], "Shape:-", encoder_input_data.shape)
print("Decoder input:", decoder_input_data[0], "Shape:-", decoder_input_data.shape)
print("Decoder output:", decoder_target_data[0], "Shape:-", decoder_target_data.shape)

Encoder input: [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]] Shape:- (10000, 4, 2862)
Decoder input: [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]] Shape:- (10000, 12, 5596)
Decoder output: [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]] Shape:- (10000, 12, 5596)


#**(Important) One-hot encode the data**

In [6]:
# This simply one-hot encodes each data of input & output, meaning, making index of the element '1' to which the character belongs
# Suppose, 'Go' corresponds to 20th character in the tokenized dictionary, then 20th index of the one-hot encoded array will be == 1

for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
  for t, char in enumerate(input_text.split()):
    encoder_input_data[i, t, input_token_index[char]] = 1.0

  # Padding the rest of the rows of the array with " "'s tokenized value
  encoder_input_data[i, t + 1 :, input_token_index[" "]] = 1.0

  for t, char in enumerate(target_text.split()):
    # decoder_target_data is ahead of decoder_input_data by one timestep (since, the first input to decoder sequence will be the input context vector)
    decoder_input_data[i, t, target_token_index[char]] = 1.0
    if t > 0:
      # decoder_target_data will be ahead by one timestep and will not include the start character.
      decoder_target_data[i, t - 1, target_token_index[char]] = 1.0

  # Padding the rest of the rows of the array with " "'s tokenized value
  decoder_input_data[i, t + 1 :, target_token_index[" "]] = 1.0
  decoder_target_data[i, t:, target_token_index[" "]] = 1.0

print("Encoder input (after modification):", encoder_input_data[0], "Shape:-", encoder_input_data.shape)
print("Decoder input (after modification):", decoder_input_data[0], "Shape:-", decoder_input_data.shape)
print("Decoder output (after modification):", decoder_target_data[0], "Shape:-", decoder_target_data.shape)

Encoder input (after modification): [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]] Shape:- (10000, 4, 2862)
Decoder input (after modification): [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]] Shape:- (10000, 12, 5596)
Decoder output (after modification): [[0. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]] Shape:- (10000, 12, 5596)


#**Creating the Model**

In [7]:
# Encoder inputs ==> shape(10000, 70) in this case. Though 'None' means that the no. of entries cannot be confirmed [since, we customize (batch_size) how many rows it's gonna process at a time]
encoder_inputs = keras.Input(shape=(None, num_encoder_tokens))

# The Encoder would be a LSTM layer of dimension = 256 but won't return output sequences
encoder = keras.layers.LSTM(latent_dim, return_state=True)

# The Encoder LSTM will output three parameters ==> outputs, hiddenStates, cellStates
encoder_outputs, state_h, state_c = encoder(encoder_inputs)

# We will only need the hiddenStates & cellStates as per the encoder's architecture
encoder_states = [state_h, state_c]

In [8]:
# Decoder inputs ==> shape(10000, 91) in this case. Though 'None' means that the no. of entries cannot be confirmed [since, we customize (batch_size) how many rows it's gonna process at a time]
decoder_inputs = keras.Input(shape=(None, num_decoder_tokens))

# The Decoder would be a LSTM layer of dimension = 256 and will return output sequences
decoder_lstm = keras.layers.LSTM(latent_dim, return_sequences=True, return_state=True)

# We won't need the decoder states for training, but are still using them since we need them for testing purpose
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)

# Finally a "Dense" layer of "softmax" activation function to find the outputs that have the highest probability of occurrence. Decoder outputs ==> shape(91)
decoder_dense = keras.layers.Dense(num_decoder_tokens, activation="softmax")
decoder_outputs = decoder_dense(decoder_outputs)

model = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)

model.compile(
    optimizer="rmsprop", loss="categorical_crossentropy", metrics=["accuracy"]
)

model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, None, 2862)     │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ input_layer_1             │ (None, None, 5596)     │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lstm (LSTM)               │ [(None, 256), (None,   │      3,193,856 │ input_layer[0][0]      │
│                           │ 256), (None, 256)]     │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lstm_1 (LSTM)             │ [(None, None, 256),    │      5,993,472 │ input_layer_1[0][0],   │
│                           │ (None, 256), (None,    │                │ lstm[0][1], lstm[0][2] │
│                           │ 256)]                  │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense (Dense)             │ (None, None, 5596)     │      1,438,172 │ lstm_1[0][0]           │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 10,625,500 (40.53 MB)

 Trainable params: 10,625,500 (40.53 MB)

 Non-trainable params: 0 (0.00 B)

#**Training the model and saving it**

In [9]:
model.fit(
    [encoder_input_data, decoder_input_data],
    decoder_target_data,
    batch_size=batch_size,
    epochs=50,
    validation_split=0.2,
)
model.save("ENG_2_FRE_Translation_Model.keras")

Epoch 1/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 12s 81ms/step - accuracy: 0.6265 - loss: 4.1710 - val_accuracy: 0.6544 - val_loss: 2.2814
Epoch 2/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 8s 60ms/step - accuracy: 0.6975 - loss: 1.9766 - val_accuracy: 0.6699 - val_loss: 2.1358
Epoch 3/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 7s 55ms/step - accuracy: 0.7146 - loss: 1.8368 - val_accuracy: 0.6914 - val_loss: 2.0066
Epoch 4/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 11s 58ms/step - accuracy: 0.7351 - loss: 1.7334 - val_accuracy: 0.7328 - val_loss: 1.9227
Epoch 5/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 10s 58ms/step - accuracy: 0.7505 - loss: 1.6628 - val_accuracy: 0.7354 - val_loss: 1.8637
Epoch 6/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 10s 53ms/step - accuracy: 0.7587 - loss: 1.5979 - val_accuracy: 0.7483 - val_loss: 1.8266
Epoch 7/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 10s 53ms/step - accuracy: 0.7612 - loss: 1.5669 - val_accuracy: 0.7458 - val_loss: 1.8015
Epoch 8/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 7s 58ms/step - accuracy: 0.7686 - loss: 1.5038 - val

#**Predicting some sample input data from the trained set of values**

In [16]:
# Load the model
model = keras.models.load_model("ENG_2_FRE_Translation_Model.keras")

# Retrieve every part of the model one by one
encoder_inputs = model.input[0]
encoder_outputs, state_h_enc, state_c_enc = model.layers[2].output
encoder_states = [state_h_enc, state_c_enc]
encoder_model = keras.Model(encoder_inputs, encoder_states)

decoder_inputs = model.input[1]
decoder_state_input_h = keras.Input(shape=(latent_dim,))
decoder_state_input_c = keras.Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_lstm = model.layers[3]
decoder_outputs, state_h_dec, state_c_dec = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs
)
decoder_states = [state_h_dec, state_c_dec]
decoder_dense = model.layers[4]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = keras.Model(
    [decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states
)

encoder_model.summary(), decoder_model.summary()

Model: "functional_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, None, 2862)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ [(None, 256), (None, 256),  │       3,193,856 │
│                                      │ (None, 256)]                │                 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 3,193,856 (12.18 MB)

 Trainable params: 3,193,856 (12.18 MB)

 Non-trainable params: 0 (0.00 B)

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1             │ (None, None, 5596)     │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ input_layer_6             │ (None, 256)            │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ input_layer_7             │ (None, 256)            │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lstm_1 (LSTM)             │ [(None, None, 256),    │      5,993,472 │ input_layer_1[0][0],   │
│                           │ (None, 256), (None,    │                │ input_layer_6[0][0],   │
│                           │ 256)]                  │                │ input_layer_7[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense (Dense)             │ (None, None, 5596)     │      1,438,172 │ lstm_1[1][0]           │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 7,431,644 (28.35 MB)

 Trainable params: 7,431,644 (28.35 MB)

 Non-trainable params: 0 (0.00 B)

(None, None)

In [24]:
# Reverse-lookup token index to decode sequences back to something readable
reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())

def decode_sequence(input_seq):
  # Encode the input as state vectors
  states_value = encoder_model.predict(input_seq, verbose=0)

  # Generate empty target sequence of length 1
  target_seq = np.zeros((1, 1, num_decoder_tokens))

  # Populate the first character of target sequence with the start character
  target_seq[0, 0, target_token_index["[START]"]] = 1.0

  # Sampling loop for a batch of sequences (to simplify, here we assume a batch of size 1)
  stop_condition = False
  decoded_sentence = ""
  while not stop_condition:
    output_tokens, h, c = decoder_model.predict(
      [target_seq] + states_value, verbose=0
    )

    # Sample a token
    sampled_token_index = np.argmax(output_tokens[0, -1, :])
    sampled_char = reverse_target_char_index[sampled_token_index]
    decoded_sentence += " " + sampled_char

    # Exit condition: either hit max length or find stop character.
    if sampled_char == "[END]" or len(decoded_sentence) > max_decoder_seq_length:
      stop_condition = True

    # Update the target sequence (of length 1).
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    target_seq[0, 0, sampled_token_index] = 1.0

    # Update states
    states_value = [h, c]

  return decoded_sentence

#**Visualizing the outputs of the input english sentences**

In [25]:
test_data = {}
for i, val in enumerate(input_texts):
  if len(test_data) < 20:
    test_data[val] = i

for key, val in test_data.items():
    input_seq = encoder_input_data[val : val + 1]
    decoded_sentence = decode_sequence(input_seq)
    print("=======================================")
    print("Input sentence:", key)
    print("Decoded sentence:", decoded_sentence)

Input sentence: Go.
Decoded sentence:  Bouge ! [END]
Input sentence: Hi.
Decoded sentence:  Cours ! [END]
Input sentence: Run!
Decoded sentence:  Cours ! [END]
Input sentence: Run.
Decoded sentence:  Cours ! [END]
Input sentence: Who?
Decoded sentence:  Qui ? [END]
Input sentence: Wow!
Decoded sentence:  À ! [END]
Input sentence: Duck!
Decoded sentence:  À ! [END]
Input sentence: Fire!
Decoded sentence:  Du ! [END]
Input sentence: Help!
Decoded sentence:  À la bonne !
Input sentence: Hide.
Decoded sentence:  Salut [END]
Input sentence: Jump!
Decoded sentence:  Saute. [END]
Input sentence: Jump.
Decoded sentence:  Saute. [END]
Input sentence: Stop!
Decoded sentence:  On ! [END]
Input sentence: Wait!
Decoded sentence:  Attends ! [END]
Input sentence: Wait.
Decoded sentence:  Attends ! [END]
Input sentence: Begin.
Decoded sentence:  Cassez-vous.
Input sentence: Go on.
Decoded sentence:  Va ! [END]
Input sentence: Hello!
Decoded sentence:  Cours ! [END]
Input sentence: Hello.
Decoded sente